<a href="https://colab.research.google.com/github/Tharindu021/Context_Based_Spell_Checker/blob/main/SpellChecker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

import NLTK is a natural language toolkit and the wordnet is a database it can use for the spell checker and also lingustic events.and from that we can get edit distance method also

In [2]:
import nltk
from nltk.corpus import wordnet
from nltk.metrics.distance import edit_distance
import pandas as pd
import numpy as np
import gensim
from gensim.models import Word2Vec
from nltk.corpus import stopwords

i try to check with in the dataset in the drive.

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# so i need to train a model using word2vec to get simillar words to the recomendation words

firstly need to get the dataset to the training the model

In [4]:
dataset_path = "/content/drive/MyDrive/Dataset/All_train_data.csv"
dp = pd.read_csv(dataset_path);
dp['input'].head()

0      I am reading score of Mahler is Symphony No . .
1    I am not interested in cars or electric applia...
2           This is my homework for my English class .
3    In comparison , Canada is catches increased an...
4    Fortunately , my older sister is friend is a d...
Name: input, dtype: object

then i need to apply preprocess before the model training

In [5]:
dp_text = dp.input.apply(gensim.utils.simple_preprocess)

after the preprocess we can tune the word2vec for our needed

In [6]:
model = gensim.models.Word2Vec(
    vector_size = 150,
    window = 10,
    workers = 6,
    min_count = 4
)

then we can train the model using the dataset the we can finalize the trainning

In [8]:
model.build_vocab(dp_text,progress_per=100)
model.train(dp_text, total_examples=model.corpus_count , epochs = 5)

(998274, 1410485)

In [9]:
model.wv.most_similar("apple")

[('dvd', 0.9918265342712402),
 ('green', 0.9915356040000916),
 ('hit', 0.9914762377738953),
 ('bridge', 0.9906883239746094),
 ('camp', 0.9901688098907471),
 ('olympics', 0.9890050888061523),
 ('upon', 0.9885343313217163),
 ('waist', 0.9882704615592957),
 ('factory', 0.988155722618103),
 ('worst', 0.9867719411849976)]

in this we can retrive the data from the dataset to use to the spell checking and then finally returns as the array

In [10]:
path = "/content/drive/MyDrive/Dataset/english Dictionary only words.csv"

df = pd.read_csv(path)
dictionary = set(df['word'].str.lower())
print(df['word'].tail())
#with open(path,'r') as file:
#    words = file.read().split()
#  return set(words)


<ipython-input-10-7a37bfac810e>:3: DtypeWarning: Columns (3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


188131    Yupon
188132      Yux
188133     Yvel
188134     Ywar
188135     Ywis
Name: word, dtype: object


punkt is a tokenizer and we need to download that and the wordnet database also then we can preprocessing our project

In [11]:
nltk.download('punkt')
#nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

I am using downloaded pretained model useto the checking the spell checking

In [56]:
def compute_similarity(embedding1, embedding2):
    return np.dot(embedding1, embedding2) / (np.linalg.norm(embedding1) * np.linalg.norm(embedding2))

# Function to find similar words to a given sentence using Word2Vec embeddings
def find_most_similar_word(sentence, input_words, word2vec_model, top_n=5):
    # Tokenize sentence and filter out words not in vocabulary
    words = [word for word in sentence.split() if word in word2vec_model.wv]
    if not words:
        return None  # Return None if no words are found in the vocabulary

    # Compute average embedding of words in the sentence
    sentence_embedding = np.mean([word2vec_model.wv[word] for word in words], axis=0)

    # Compute similarity between sentence embedding and embeddings of input words
    similarities = [(input_word, compute_similarity(sentence_embedding, word2vec_model.wv[input_word]))
                    for input_word in input_words if input_word in word2vec_model.wv]

    # Sort words by similarity in descending order
    similarities.sort(key=lambda x: x[1], reverse=True)

    # Return top n similar words
    return similarities[:top_n]

from below code we can get the input sentence and that sentence can tokenize using nltk.word_tokenize then we need to chech that words one by one then if it is not in the dataset then we can print there is no word like user input and show then to the what word are the mispelled and the recomendations using edit distance algorithm

In [54]:
def spell_checker(sentence, dictionary):
      simillar_word = []
      words = nltk.word_tokenize(sentence)
      for word in words:
          if word.lower() not in dictionary:
             simillar_word = recomend_simillar_words(word.lower())
             #print(word,simillar_word,sentence)
             recomended_words = find_most_similar_word(sentence,simillar_word,model)
             print("Recommended words for '{}' are: {}".format(word, recomended_words))

in this code we can get the simmilar words for misspelled words that are input have i choose only below 2 distance. and then recomend to the client only 10 words.

In [14]:
def recomend_simillar_words(words):
  simillar_words = []
  for dict_words in dictionary:
    distance = edit_distance( dict_words , words )
    if distance <= 1:
      simillar_words.append( (dict_words , distance) )
  simillar_words.sort(key=lambda x : x[1])
  return [ word[0] for word in simillar_words[:20]]

in that code part we can input the sentence then it strictly move to the spell_checker code part then it can check whether the word is in the dataset or not

In [55]:

user_input = ""
while user_input.lower() != "exit":
    user_input = input("Please enter the sentence you want or type 'exit' to quit: ")
    if user_input.lower() == "exit":
        print("Exiting...")
    else:
        spell_checker(user_input, dictionary)

Please enter the sentence you want or type 'exit' to quit: i eat aple
Recommended words for 'aple' are: [('apple', 0.70152116), ('able', 0.6632011)]
Please enter the sentence you want or type 'exit' to quit: exit
Exiting...
